# PACベイズを使った制御手法

参考：
* [PAC-Bayes Control: Learning Policies that Provably Generalize to Novel Environments]()

作ったロボットが手持ちのデータセットでうまく動くことを確認できたとしましょう．
このロボットは，見たこと無い環境でどれくらい良く動くことができるでしょうか？

未知の環境でも動くように保証する制御手法は色々あります（例えばH-∞制御やChance-constraintなどのロバスト制御）．
ここでは，PACベイズに基づいた制御手法を学んでみましょう．


## 設定と表記

* 環境のモデル：$x(t+1)=f(x(t), u(t); E)$．
    * ここで$t$は時間のインデックス
    * $x$と$u$は状態と制御入力．
    * $E$は「一般の環境」を表す．環境の集合を$\mathcal{E}$とする．
* 環境の集合上に，未知の分布$\mathcal{D}$が存在し，そこから環境がサンプルされるとする．
    * $S=\{E_1, \dots, E_N\}$を$N$個の環境のデータセットとする．
* $g: \mathcal{X}\times \mathcal{E}\to\mathcal{Y}$を状態からロボットのセンサーへの写像とする．つまり，$y=g(x;E)\in \mathcal{Y}$．
    * $\pi : \mathcal{Y} \to \mathcal{U}$を制御方策とする．
* $r_\pi: \mathcal{E}\to (\mathcal{X}\times \mathcal{U})^T$を方策$\pi$によって環境をロールアウトして得られる結果とする．
    * 簡単のため，環境とは確率的な影響を全て含有した存在であるとして，ロールアウト関数は決定的であるとする．
* $C(r_\pi;E) \in [0, 1]$を環境$E$で$T$時刻ロールアウトした結果得られるコストとする．
    * これは環境が与えられたら簡単に計算できるとする．例えばシミュレータなどでは簡単．

この問題設定で，次のゴールを達成することを考えよう．

$$
C^* := \min_{P \in \mathcal{P}} C_{\mathcal{D}}(P) := \min_{P\in \mathcal{P}} \mathbb{E}_{E\sim \mathcal{D}} \mathbb{E}_{\pi\sim P} [C(r_\pi;E)]
$$

ここで，$P$は方策集合上の分布とした．

<!-- $$
\min_{\pi \in \Pi} C_{\mathcal{D}}(\pi) := \min_{\pi \in \Pi} \mathbb{E}_{E\sim \mathcal{D}} [C(r_\pi;E)]
$$ -->



## 教師あり学習でのPACベイズ

教師あり学習でのPACベイズを軽く復習しておきましょう．
* $\mathcal{Z}$を入力空間，$\mathcal{Z}'$をラベルの集合とします．
* $\mathcal{D}$を入力空間上の未知の分布とします．
* $\mathcal{H}$を$w\in \mathbb{R}^d$によってパラメータ化された仮説集合とし，仮説を$h_w: \mathcal{Z}\to \mathcal{Z}'$とします．
* $l:\mathcal{H}\times \mathcal{Z}\to \mathbb{R}$をロス関数とします．
* $\mathcal{P}$をパラメータ空間上の分布とします．

PACベイズによる解析は次のような設定で一般に使用できます．
1. データを観測する前に，事前分布$P_0\in \mathcal{P}$が存在している．
2. 学習用のデータサンプル$S=\{z_i\}_{i=1}^N$が存在し，事後分布$P\in \mathcal{P}$を選択する．この事後分布はデータと事前分布の両方に依存する．

事後分布の学習損失を
$$
l_S(P):= \frac{1}{N} \sum_{z\in S} \mathbb{E}_{w\in P}[l(h_w;z)]
$$
として，真の期待損失を
$$
l_\mathcal{D}(P):= \mathbb{E}_{z\in \mathcal{D}} \mathbb{E}_{w\in P}[l(h_w;z)]
$$
とする．

このとき，教師あり学習では次のPACベイズバウンドが成立します：

---
確率$1-\delta$以上で，

$$
\mathbb{D}(l_S(P)\| l_D(P)) \leq \frac{\mathbb{D}(P\|P_0)+\log(\frac{2\sqrt{N}}{\delta})}{N}
$$

が成立します．
ここで，スカラー$p, q \in [0, 1]$に対して

$$
\mathbb{D}(p\|q) := \mathbb{D}(B(p)\|B(q))=p\log \frac{p}{q} + (1-p)
$$
としました．
直感的には，このバウンドは$l_D(P)$と$l_S(P)$がどれだけ近いか？を表しています．

ここで，KLダイバージェンスの逆を取ると，
$$
l_D(P) \leq \mathbb{D}^{-1}\left(l_S(P) \| \frac{\mathbb{D}(P\|P_0)+\log(\frac{2\sqrt{N}}{\delta})}{N}\right)
$$
が得られます．さらにPinskerの不等式を使えば，次のようなバウンドに変形できます

$$
l_{\mathcal{D}}(P) \leq l_S(P) + \sqrt{\frac{\mathbb{D}(P\|P_0)+\log(\frac{2\sqrt{N}}{\delta})}{2N}}
$$

つまり，期待損失は，経験損失と「正則化項」の和で上からバウンドできます．
この右辺を小さくするような仮説上の分布$P$を選択すれば，期待損失を小さくすることができます．

---


## 制御問題でのPACベイズ

上で設定した制御問題と教師あり学習を次のように紐付ければ，制御問題でもPACベイズを使うことができます．

* 教師あり学習：入力 $z\in \mathcal{Z}$，仮説 $h_w: \mathcal{Z}\to \mathcal{Z}'$, 損失 $l(h_w;z)$
* 制御問題：環境 $E\in \mathcal{E}$，ロールアウト関数 $r_\pi: \mathcal{E}\to (\mathcal{X}\times \mathcal{U})^H$, コスト $C(r_\pi;E)$

ここで，方策は$w\in \mathbb{R}^d$でパラメータ化されているとしましょう．
$P$をパラメータ空間上の事後分布，$P_0$を事前分布として，経験コストを　

$$
C_S(P) := \frac{1}{N}\sum_{E\in S} \mathbb{E}_{w\sim P}[C(r_w;E)]
$$

と書きましょう．このとき，次のPACバウンドが成立します．

$$
C_{\mathcal{D}}(P) \leq C_{PAC}(P) := C_S(P) + \sqrt{\frac{\mathbb{D}(P\|P_0)+\log(\frac{2\sqrt{N}}{\delta})}{2N}}
$$

こうすると，$C_{PAC}(P)$を最小化する枠組みを考えれば，期待コストを最小化できそうです．